In [1]:
import os
import math
from tarefa02.gcrf import Gcrf, Point, Edge, Face

In [2]:
gc = Gcrf()

In [3]:
p = Point(1, 2, 3)
print(p)

(1, 2, 3)


In [4]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)

e = Edge(p1, p2)

print(e)

[(1, 2, 3) -> (4, 5, 6)]


In [5]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)
p3 = Point(7, 8, 9)

e1 = Edge(p1, p2)
e2 = Edge(p2, p3)
e3 = Edge(p3, p1)

f = Face(e1, e2, e3)
f_ = Face(e3, e1, e2)
print(f)

{[(1, 2, 3) -> (4, 5, 6)], [(4, 5, 6) -> (7, 8, 9)], [(7, 8, 9) -> (1, 2, 3)]}


In [6]:
def checa_face(F, f):
    for face in F:
        if f == face:
            return True
    return False

In [7]:
p1 = Point(0,-1,0)
p2 = Point(0, 0, 0)
p3 = Point(0,0,1)
a = Point(1, 0,0)
b = Point(-1,0,0)
print(gc.signed_volume(p1,p2,p3,a))
print(gc.signed_volume(p1,p2,p3,b))

0.16666666666666666
-0.16666666666666666


In [8]:
def PivotAroundEdge(f, e, P):
    minAngle = 2*math.pi
    candidate = None
    print("E",e, "F",f)
    n1 = normal(f.e1.p1, f.e1.p2, f.e2.p2)
    
    for i in range(len(P)):
        if P[i] == e.p1 or P[i] == e.p2: continue
        print("PONTOS DA FACE",e.p2, e.p1, P[i])
        n2 = normal(e.p2, e.p1, P[i])
        print("n1", n1, "n2", n2)
#         if n2 == Point(0,0,0): continue
        angle = -1*gc.angulo(n1,n2)
        print("Angulo",angle)
        if angle < minAngle and P[i] != e.p1 and P[i] != e.p2:
            minAngle = angle
            candidate = i
    return P[candidate]

In [9]:
# def PivotAroundEdge(e, P):
#     minAngle = 2*math.pi
#     candidate = None
#     normal = gc.prod_vetorial(e.p1, e.p2)
# #     print("NORMAL:", normal)
#     for i in range(len(P)):
#         print("P[i]",P[i])
#         angle = gc.angulo(normal, P[i])
#         if angle < minAngle:
#             minAngle = angle
#             candidate = i
#     return P[candidate]

In [10]:
def BottomMostLeftMostBackMost(P):
    ys = [p.y for p in P]
    min_value = min(ys)
    min_index = ys.index(min_value)
    return P[min_index]

In [11]:
def normal(p1, p2, p3):
    v1 = gc.subtr_vetorial(p1,p2)    
    v2 = gc.subtr_vetorial(p1,p3)
    return gc.prod_vetorial(v1,v2)

In [12]:
def find_first_face(P):
    minAngle = 2*math.pi
    candidate = None
    
    p1 = BottomMostLeftMostBackMost(P)
    p2 = Point(p1.x, p1.y, p1.z-1)
    p3 = Point(p1.x+1, p1.y, p1.z) 
    print("primeiro P1", p1, "P2", p2, "P3", p3)
    n1 = normal(p1, p2, p3)
    
    for i in range(len(P)):
        if P[i] == p1: continue
        n2 = normal(p1, p2, P[i])
        angle = gc.angulo(n1,n2)
        if angle < minAngle and P[i] != p:
            minAngle = angle
            candidate = i
        
    p3 = P[candidate]

    
    n1 = normal(p1, p2, p3)
    for j in range(len(P)):
        if P[j] == p1 or P[j] == p3: continue
        n2 = normal(p1, P[j], p3)
        angle = gc.angulo(n1,n2)
        if angle < minAngle and P[j] != p1 and P[j] != p3:
            minAngle = angle
            candidate = j
    
    p2 = P[candidate]
    
    e1 = Edge(p1,p2)
    e2 = Edge(p2, p3)
    e3 = Edge(p3, p1)
    print("P {} Q {} R {}".format(p1,p2,p3))
    face = Face(e1, e2, e3)
    return face

In [13]:
def NotProcessed(E, e):
#     print("Aresta no Not Processed",e)
    for i in E:
        if e == i:
            return False
    return True

In [14]:
def GiftWrap(P):
#     t = FindTriangleOnHull(P)
    t = find_first_face(P)
    print("Triangulo encontrado:", t)
    Q = [t.e1, t.e2, t.e3]
    H = [t]
    Processed = []
    print("Q possui as arestas:")
    for q in Q: print(q)
    while not len(Q) == 0:
        e = Q.pop(0)
#         e = Edge(e.p2, e.p1)
        print("Aresta a ser processada:", e)
        if NotProcessed(Processed, e):
            print("Aresta {} não foi processada, entrando no IF".format(e))
            q = PivotAroundEdge(t, e, P)
            print("Resultado do PivotAroundEdge:", q)
            t = Face(e, Edge(e.p2, q), Edge(q, e.p1))
            print("Nova face:", t)
            face_no_hull = checa_face(H, t)
            if not face_no_hull:
                H.append(t)
                print("{} entrou no Hull".format(t))
            else: print("Face já está no Hull")
#             e_ = Edge(e.p2, e.p1)
            Q.extend([t.e1, t.e2, t.e3])
            print("Q agora possui as arestas:")
            for q in Q: print(q)
            Processed.append(e)
            print("Processed agora inclui a aresta {} e contém as seguintes arestas:".format(e))
            for aresta in Processed: print(aresta)
        else: print("Aresta JÁ PROCESSADA")
    return H

In [15]:
# p1 = Point(-3, 0, 0)
# p2 = Point(0, -3, 0)
# p3 = Point(0, 2, 0)
# p4 = Point(0, 0, 3)
# p5 = Point(0, 0, -4)
# p6 = Point(4, 0, 0)

p1 = Point(-1, 0, 0)
p2 = Point(0, -1, 0)
p3 = Point(0, 1, 0)
p4 = Point(0, 0, 1)
p5 = Point(0, 0, -1)
p6 = Point(1, 0, 0)
# p7 = Point(0,0,0)

# P = [p1, p2, p3, p4, p5, p6, p7]
P = [p1, p2, p3, p4, p5, p6]

H = GiftWrap(P)
for h in H:
    print(h)
print(len(H))

primeiro P1 (0, -1, 0) P2 (0, -1, -1) P3 (1, -1, 0)
P (0, -1, 0) Q (0, 0, 1) R (-1, 0, 0)
Triangulo encontrado: {[(0, -1, 0) -> (0, 0, 1)], [(0, 0, 1) -> (-1, 0, 0)], [(-1, 0, 0) -> (0, -1, 0)]}
Q possui as arestas:
[(0, -1, 0) -> (0, 0, 1)]
[(0, 0, 1) -> (-1, 0, 0)]
[(-1, 0, 0) -> (0, -1, 0)]
Aresta a ser processada: [(0, -1, 0) -> (0, 0, 1)]
Aresta [(0, -1, 0) -> (0, 0, 1)] não foi processada, entrando no IF
E [(0, -1, 0) -> (0, 0, 1)] F {[(0, -1, 0) -> (0, 0, 1)], [(0, 0, 1) -> (-1, 0, 0)], [(-1, 0, 0) -> (0, -1, 0)]}
PONTOS DA FACE (0, 0, 1) (0, -1, 0) (-1, 0, 0)
n1 (-1, -1, 1) n2 (1, 1, -1)
Angulo 1.0000000000000002
PONTOS DA FACE (0, 0, 1) (0, -1, 0) (0, 1, 0)
n1 (-1, -1, 1) n2 (2, 0, 0)
Angulo 0.5773502691896258
PONTOS DA FACE (0, 0, 1) (0, -1, 0) (0, 0, -1)
n1 (-1, -1, 1) n2 (2, 0, 0)
Angulo 0.5773502691896258
PONTOS DA FACE (0, 0, 1) (0, -1, 0) (1, 0, 0)
n1 (-1, -1, 1) n2 (1, -1, 1)
Angulo -0.33333333333333337
Resultado do PivotAroundEdge: (1, 0, 0)
Nova face: {[(0, -1, 0) -> 

Angulo 0.5773502691896258
PONTOS DA FACE (0, 0, -1) (0, -1, 0) (0, 1, 0)
n1 (2, 0, 0) n2 (-2, 0, 0)
Angulo 1.0
PONTOS DA FACE (0, 0, -1) (0, -1, 0) (0, 0, 1)
n1 (2, 0, 0) n2 (-2, 0, 0)
Angulo 1.0
PONTOS DA FACE (0, 0, -1) (0, -1, 0) (1, 0, 0)
n1 (2, 0, 0) n2 (-1, 1, 1)
Angulo 0.5773502691896258
Resultado do PivotAroundEdge: (-1, 0, 0)
Nova face: {[(0, -1, 0) -> (0, 0, -1)], [(0, 0, -1) -> (-1, 0, 0)], [(-1, 0, 0) -> (0, -1, 0)]}
Face já está no Hull
Q agora possui as arestas:
[(0, 0, -1) -> (-1, 0, 0)]
[(0, 0, 1) -> (1, 0, 0)]
[(1, 0, 0) -> (0, -1, 0)]
[(0, -1, 0) -> (0, 0, 1)]
[(1, 0, 0) -> (0, -1, 0)]
[(0, -1, 0) -> (0, 0, -1)]
[(0, 0, -1) -> (1, 0, 0)]
[(-1, 0, 0) -> (0, 0, -1)]
[(0, 0, -1) -> (0, -1, 0)]
[(0, -1, 0) -> (-1, 0, 0)]
[(0, 0, -1) -> (0, 0, 1)]
[(0, 0, 1) -> (0, -1, 0)]
[(0, -1, 0) -> (0, 0, -1)]
[(0, -1, 0) -> (0, 0, -1)]
[(0, 0, -1) -> (-1, 0, 0)]
[(-1, 0, 0) -> (0, -1, 0)]
Processed agora inclui a aresta [(0, -1, 0) -> (0, 0, -1)] e contém as seguintes arestas:
[(0, 

In [16]:
def write_faces(F, P, filename=False):
    i1 = 0
    i2 = 0
    i3 = 0
    if not filename:
        filename = os.path.join(os.getcwd(), 'faces.obj')
    with open(filename, 'w') as f:
        for point in P:
            f.write("v {} {} {}".format(point.x, point.y, point.z))
            f.write('\n')
        f.write('\n')
        for i in range(len(F)):
            for j in range(len(P)):
                if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                    i1 = j
                if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                    i2 = j
                if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                    i3 = j
            f.writelines("f {} {} {}".format(i1+1, i2+1, i3+1))
            f.write('\n')


In [17]:
def print_faces(F, P):
    i1 = 0
    i2 = 0
    i3 = 0
    for i in range(len(F)):
        for j in range(len(P)):
            if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                i1 = j
            if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                i2 = j
            if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                i3 = j
                
        print("f {} {} {}".format(i1+1, i2+1, i3+1))

In [18]:
def read_vertices_from_obj(objPath):
    points = []
    with open(objPath, 'r') as f:
        for line in f:
            line = line.replace('v', '')
            line_ = line.split()
            points.append(Point(float(line_[0]), float(line_[1]), float(line_[2])))
    return points

In [19]:
points = read_vertices_from_obj(r"C:\Users\romul\Documents\UFC\MESTRADO\Disciplinas\Geometria Computacional\super-nintendo-blend\p1.obj")
print(len(points))

348


In [20]:
path = os.path.join(os.getcwd(), 'obj')
if os.path.isdir(path):
    print("OBJ folder exists")
else:
    print("Creating OBJ folder.")
    os.mkdir(path)

OBJ folder exists


In [21]:
nH = GiftWrap(points)
# for h in nH:
#     print(h)
print(len(nH))

primeiro P1 (-0.258873, 0.032391, 0.264508) P2 (-0.258873, 0.032391, -0.735492) P3 (0.741127, 0.032391, 0.264508)
P (-0.258873, 0.032391, 0.264508) Q (-0.346055, 0.032391, 0.129129) R (-0.346055, 0.032391, 0.129129)
Triangulo encontrado: {[(-0.258873, 0.032391, 0.264508) -> (-0.346055, 0.032391, 0.129129)], [(-0.346055, 0.032391, 0.129129) -> (-0.346055, 0.032391, 0.129129)], [(-0.346055, 0.032391, 0.129129) -> (-0.258873, 0.032391, 0.264508)]}
Q possui as arestas:
[(-0.258873, 0.032391, 0.264508) -> (-0.346055, 0.032391, 0.129129)]
[(-0.346055, 0.032391, 0.129129) -> (-0.346055, 0.032391, 0.129129)]
[(-0.346055, 0.032391, 0.129129) -> (-0.258873, 0.032391, 0.264508)]
Aresta a ser processada: [(-0.258873, 0.032391, 0.264508) -> (-0.346055, 0.032391, 0.129129)]
Aresta [(-0.258873, 0.032391, 0.264508) -> (-0.346055, 0.032391, 0.129129)] não foi processada, entrando no IF
E [(-0.258873, 0.032391, 0.264508) -> (-0.346055, 0.032391, 0.129129)] F {[(-0.258873, 0.032391, 0.264508) -> (-0.3460

ZeroDivisionError: float division by zero

In [ ]:
print_faces(H, P)
write_faces(H, P, path+"/bento.obj")

In [ ]:
write_faces(nH, points, path+"/controle.obj")
print_faces(nH, points)
